In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import re
import os
import pandas as pd
import time
import lxml
import traceback
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import shutil
from datetime import datetime


import warnings
warnings.filterwarnings('ignore')

In [4]:
reports_df = pd.DataFrame(columns=['industry', 'title', 'rating', 'rating_alter', 'broker', 'date', 'summary'])

In [5]:
reports_df

,industry,title,rating,rating_alter,broker,date,summary


In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)

url = "https://data.eastmoney.com/report/hyyb.html"
driver.get(url)

In [8]:
def get_page_num(num):
        """模拟鼠标点击下一页"""

        # （1）定位到表格下面页码选择区域，定位元素："macresearch_table_pager"
        element_page = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "industry_table_pager")))

        # （2）获取搜索页面的按钮"ipt"
        tr_options = element_page.find_element_by_class_name("ipt")

        # （3）填入页码，模拟点击鼠标
        tr_options.clear()  # 必须清空当前的搜索页码
        tr_options.send_keys('{}'.format(str(num)))
        element_page.find_element_by_class_name("btn").click()

        # （4）点击后必须等待固定时长
        time.sleep(10)

In [10]:
def download_report(text_link):

        # 获取页面内容
        orihtml = requests.get(text_link).content
        soup = BeautifulSoup(orihtml, "lxml")

        # 判断报告是否是空页
        if soup.find('div', class_='ctx-content') == None:
            return None

        page_con = []
        for a in soup.find('div', class_='ctx-content').find_all('p'):
            page_con.append(str(a.text))
            
        return "\n".join(page_con)

In [12]:
def get_report_date(start_date, end_date, pageNum=1):
        """ 以起始和终止时期为爬取标准 """

        # 转化为可比较大小的日期
        start_date = time.strptime(start_date, "%Y-%m-%d")
        end_date = time.strptime(end_date, "%Y-%m-%d")

        # 每次指定页码
        pageNum_init = pageNum
        FLAG = True
        while FLAG:
            get_page_num(pageNum_init)
            print(f"Page number: {pageNum_init}")
            try:
                element_table = WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.ID, "industry_table")))
                tr_options = element_table.find_elements_by_tag_name("tr")  # 获取所有的tr标签，"tr"

                # 再遍历td标签
                for tr_option in tr_options:
                    # 获得：序号、报告名称、作者、机构名称、近一月机构宏观研报数量、日期
                    td_options = tr_option.find_elements_by_tag_name("td")  # 定位元素"td"
                    re_sum_info = []
                    for td_option in td_options:
                        re_sum_info.append(td_option.text)
                    if not re_sum_info:
                        continue

                    print("page:", re_sum_info)
                        
                    # 日期判断
                    time_tmp = time.strptime(re_sum_info[-1], "%Y-%m-%d")
                    if time_tmp < start_date:
                        FLAG = False
                        break
                    elif time_tmp > end_date:
                        continue
                    else:
                        url_element = td_options[4]
                        if not url_element:
                            continue
                        link = url_element.find_elements_by_xpath(".//*[@href]")[0]  # 获取链接
                        text_link = link.get_attribute('href')
                        summary  = download_report(text_link)
                        
                        industry     = re_sum_info[1]
                        title        = re_sum_info[4]
                        rating       = re_sum_info[5]
                        rating_alter = re_sum_info[6]
                        broker       = re_sum_info[7]
                        date         = re_sum_info[-1]
                        
                        reports_df.loc[len(reports_df.index)] = [industry, title, rating, rating_alter, broker, date, summary]
                        
            except Exception as e:
                info = traceback.format_exc()
                print(info)
                pageNum_init += 1
            pageNum_init += 1

In [13]:
get_report_date('2022-05-26', '2022-05-27', 1)
driver.quit()

Page number: 1
page: ['1', '食品饮料', '-0.01%', '详细  资金流  股吧  专区', '国元新食饮每日一图：燕麦奶市场，孕育中', '-', '-', '国元证券', '143', '2022-05-27']
page: ['2', '通信设备', '0.03%', '详细  资金流  股吧  专区', '通信行业重大事项点评：深圳发布促消费若干措施，关注背后通信板块机会', '增持', '维持', '国信证券', '67', '2022-05-27']
page: ['3', '石油行业', '3.10%', '详细  资金流  股吧  专区', '每日原油：英国将向油气企业征收临时暴利税', '-', '-', '信达证券', '43', '2022-05-27']
page: ['4', '化肥行业', '1.90%', '详细  资金流  股吧  专区', '基础化工行业深度报告：当前怎么看农化板块？', '持有', '维持', '东方证券', '6', '2022-05-27']
page: ['5', '航天航空', '-0.13%', '详细  资金流  股吧  专区', '国防装备行业点评报告：中航电子拟吸收合并中航机电，军工“内生+外延”双驱动逻辑强化', '持有', '无', '浙商证券', '64', '2022-05-27']
page: ['6', '石油行业', '3.10%', '详细  资金流  股吧  专区', '石油化工行业研究：警惕油品飙涨风险！', '买入', '调高', '国金证券', '43', '2022-05-27']
page: ['7', '航空机场', '0.86%', '详细  资金流  股吧  专区', '国内客运航班运行财政补贴政策点评：夯实民航安全基础，推动航司“应飞尽飞”', '增持', '维持', '光大证券', '12', '2022-05-27']
page: ['8', '造纸印刷', '0.10%', '详细  资金流  股吧  专区', '造纸：底部掘金，造纸行业行至何处？', '增持', '维持', '国盛证券', '35', '2022-05-27']
page: ['9', '文化传媒', '-0.27%', '详细  资金流  股吧  专区'

page: ['69', '有色金属', '0.05%', '详细  资金流  股吧  专区', '有色金属行业深度报告：缺口持续扩大，电解铝行业高度景气', '持有', '维持', '东亚前海证券', '113', '2022-05-26']
page: ['70', '光伏设备', '-0.16%', '详细  资金流  股吧  专区', '光伏行业2021年报和2022年1季报总结：市场需求旺盛，业绩加速增长', '增持', '维持', '中原证券', '56', '2022-05-26']
page: ['71', '游戏', '-0.75%', '详细  资金流  股吧  专区', '游戏行业数据周报：网易《暗黑破坏神：不朽》国内定档6月23日', '增持', '维持', '德邦证券', '6', '2022-05-26']
page: ['72', '商业百货', '0.06%', '详细  资金流  股吧  专区', '4月美妆电商数据跟踪报告：4月美妆行业线上销售承压，但部分头部公司表现亮眼', '增持', '维持', '万联证券', '85', '2022-05-26']
page: ['73', '通信设备', '0.00%', '详细  资金流  股吧  专区', '通信区块链行业深度：大转换——以太坊合并，从显卡跌价说起', '增持', '维持', '国盛证券', '67', '2022-05-26']
page: ['74', '通用设备', '0.53%', '详细  资金流  股吧  专区', '机械设备工业气体行业报告：行业发展稳定且迅速，成长前景美好', '增持', '维持', '天风证券', '65', '2022-05-26']
page: ['75', '环保行业', '0.24%', '详细  资金流  股吧  专区', '动保行业：成长持续+周期催化，布局正当时——行业投资框架与标的全面梳理', '-', '-', '国盛证券', '51', '2022-05-26']
page: ['76', '生物制品', '0.40%', '详细  资金流  股吧  专区', '医药生物行业新冠产业链更新报告：多管齐下，恢复可期', '持有', '无', '浙商证券', '28', '2022-05-26']
page: ['77'

In [14]:
reports_df

,industry,title,rating,rating_alter,broker,date,summary
0,食品饮料,国元新食饮每日一图：燕麦奶市场，孕育中,-,-,国元证券,2022-05-27,
1,通信设备,通信行业重大事项点评：深圳发布促消费若干措施，关注背后通信板块机会,增持,维持,国信证券,2022-05-27,事项：\r\n 深圳市发展和改革委员会、 深圳市商务局、 深圳市工业和信息化局、 深圳...
2,石油行业,每日原油：英国将向油气企业征收临时暴利税,-,-,信达证券,2022-05-27,新闻：\r\n 1、英国政府26日宣布，将向石油和天然气企业征收“暴利税”，作为缓解民...
3,化肥行业,基础化工行业深度报告：当前怎么看农化板块？,持有,维持,东方证券,2022-05-27,化肥——全球供需收紧，能源通胀加持：磷、钾三大肥全球供需紧平衡，需求方面随着疫情后的复苏...
4,航天航空,国防装备行业点评报告：中航电子拟吸收合并中航机电，军工“内生+外延”双驱动逻辑强化,持有,无,浙商证券,2022-05-27,核心观点\n 中航电子、中航机电合并打造航空机载龙头，军工央企“外延”注入加速\n ...
...,...,...,...,...,...,...,...
83,汽车零部件,2022年4月新能源汽车行业运行数据报告：汽车行业遭受疫情冲击，新能源汽车增长保持韧性,-,-,中国信通院,2022-05-26,2022年4月份，疫情对我国汽车行业的冲击影响在供给端和市场端同步显现，上期报告对新能源...
84,纺织服装,纺织服装行业深度报告：从巨子生物看重组胶原蛋白行业,持有,维持,东方证券,2022-05-26,重组胶原蛋白发展潜力巨大，未来增速预计大幅超越传统动物源胶原蛋白。胶原蛋白是一种护肤及皮...
85,电子元件,2022年4月电子行业运行数据报告：电子制造业成为疫情下工业经济增长的重要支柱,-,-,中国信通院,2022-05-26,2022年4月，我国电子制造业保持了在制造业中的领先地位，生产与投资增速持续高于工业总体...
86,保险,保险Ⅱ行业深度报告：偿二代二期工程季报评价体系（财险篇）：拆解承保效益结构，剖析资本经营效率,增持,维持,东吴证券,2022-05-26,投资要点 \n 我们建立了【东吴证券-偿二代二期工程指标评价体系（财险篇）】， 通过标...


In [15]:
data_dict = reports_df[::-1].to_dict("records")


In [16]:
data_dict

[{'industry': '纺织服装',
  'title': '纺织服饰行业+公司首次覆盖：消费升级与疫情催化，双轮驱动露营风起',
  'rating': '增持',
  'rating_alter': '维持',
  'broker': '东吴证券',
  'date': '2022-05-26',
  'summary': '\u3000\u3000投资要点\n\u3000\u3000户外风起，疫情下露营领跑增速。 中国户外行业 2021 年呈现复苏态势，近年来多个项目陆续兴起，继路跑、冰雪之后，露营于 2020 年以来崭露头角、尤其 22 年 3 月份以来热度直线上升，主要受益于疫情催化精致露营异军突起。不同于以户外生存为特点的野外露营，精致露营主打轻奢休闲，参与门槛低、活动场景丰富、受众广泛，推动露营从小众项目走入大众视野，20 年疫情之后热度不断高涨，露营相关企业注册数量猛增、露营地市场规模高速增长、网络关注度不断攀升、一级市场融资增多。\n\u3000\u3000增长动力：消费升级与疫情催化双轮驱动。 1）短期疫情催化： 疫情下境内跨省市、出境游难以成行，露营场景局限性较低、住宿更为自由， 既能亲近自然、又能保持充足社交安全距离， 契合疫情下消费者出行偏好，推动短途露营热度高涨， 22 年新一轮疫情爆发进一步刺激消费。参考海外，疫情对北美露营活动参与率提升催化作用同样明显，且 21 年防疫政策放松后热度依旧不减。 2）长期消费力提升奠定发展基础： 户外休闲活动开展与国民经济水平相关性强，参考日本经验， 1984 年日本人均 GDP 超 1 万美元后露营行业快速发展，至 1996 年参与人数年均复合增速达 12%。 2019 年我国人均 GDP 首超 1 万美元、 为露营行业发展奠定经济基础。预计随消费者教育及习惯养成，露营有望成为一种生活方式、并在疫后仍将获得持续发展。 3）社媒传播、政策驱动加速行业发展： 露营参与者以对社媒依赖性高的年轻群体为主，近期露营在社媒高度传播助推风潮兴起。 14 年以来户外露营相关支持政策持续发布，夯实行业发展基础。\n\u3000\u3000未来空间：中国露营行业潜力亟待挖掘，成长空间足。 露营行业前景广阔，未来增长动力主要来自两个方面： 1）渗透率提升： 目前中国露营行业渗透率约 1%， 对标美日增长空间超 10 